In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)


In [2]:
df = pd.read_csv('./smoke_detection.csv')
df.head()

,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire
0,20.000,57.36,0,400,12306,18520,939.735,0.0,0.0,0.0,0.0,0.0,0,0
1,20.015,56.67,0,400,12345,18651,939.744,0.0,0.0,0.0,0.0,0.0,1,0
2,20.029,55.96,0,400,12374,18764,939.738,0.0,0.0,0.0,0.0,0.0,2,0
3,20.044,55.28,0,400,12390,18849,939.736,0.0,0.0,0.0,0.0,0.0,3,0
4,20.059,54.69,0,400,12403,18921,939.744,0.0,0.0,0.0,0.0,0.0,4,0


In [3]:
# train and test data

train = df.sample(frac=0.9, replace=False, random_state=1)
test = df.drop(train.index)

train.shape,test.shape

((56367, 14), (6263, 14))

In [4]:
#storing in np arrays

xTrain = np.array(train.drop(columns=['Fire']))
yTrain = np.array(train['Fire']).reshape((-1,1))


xTest = np.array(test.drop(columns=['Fire']))
yTest = np.array(test['Fire']).reshape((-1,1))

print(f'x_train = {xTrain.shape}')
print(f'y_train = {yTrain.shape}')
print(f'x_test = {xTest.shape}')
print(f'y_test = {yTest.shape}')

x_train = (56367, 13)
y_train = (56367, 1)
x_test = (6263, 13)
y_test = (6263, 1)


In [5]:
#normalize
samples = xTrain.shape[1]
for i in range(samples):
    print(f"Column {i} Max, Min pre normalization: {np.max(xTrain[:,i]):0.2f}, {np.min(xTrain[:,i]):0.2f}")

norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(xTrain)  # learns mean, variance
xTrainN = norm_l(xTrain)

for i in range(samples):
    print(f"Column {i} Max, Min post normalization: {np.max(xTrainN[:,i]):0.2f}, {np.min(xTrainN[:,i]):0.2f}")


Column 0 Max, Min pre normalization: 59.93, -22.01
Column 1 Max, Min pre normalization: 75.20, 10.74
Column 2 Max, Min pre normalization: 60000.00, 0.00
Column 3 Max, Min pre normalization: 60000.00, 400.00
Column 4 Max, Min pre normalization: 13803.00, 10668.00
Column 5 Max, Min pre normalization: 21410.00, 15317.00
Column 6 Max, Min pre normalization: 939.86, 930.85
Column 7 Max, Min pre normalization: 14333.69, 0.00
Column 8 Max, Min pre normalization: 45432.26, 0.00
Column 9 Max, Min pre normalization: 61482.03, 0.00
Column 10 Max, Min pre normalization: 51914.68, 0.00
Column 11 Max, Min pre normalization: 30026.44, 0.00
Column 12 Max, Min pre normalization: 24993.00, 0.00
Column 0 Max, Min post normalization: 3.07, -2.65
Column 1 Max, Min post normalization: 3.02, -4.29
Column 2 Max, Min post normalization: 7.52, -0.25
Column 3 Max, Min post normalization: 31.95, -0.14
Column 4 Max, Min post normalization: 3.18, -8.42
Column 5 Max, Min post normalization: 2.73, -7.32
Column 6 Max,

In [6]:
Xt = np.tile(xTrainN,(4,1))
Yt= np.tile(yTrain,(4,1))   
print(Xt.shape, Yt.shape) 

(225468, 13) (225468, 1)


## Tensorflow Model

In [7]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        tf.keras.Input(shape=(13,)),
        Dense(20, activation='sigmoid', name = 'layer1'),
        Dense(10, activation='sigmoid', name = 'layer2'),
        Dense(5, activation='sigmoid', name = 'layer3'),
        Dense(2, activation='sigmoid', name = 'layer4'),
        Dense(1, activation='sigmoid', name = 'layer5'),
     ]
)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 20)                280       
                                                                 
 layer2 (Dense)              (None, 10)                210       
                                                                 
 layer3 (Dense)              (None, 5)                 55        
                                                                 
 layer4 (Dense)              (None, 2)                 12        
                                                                 
 layer5 (Dense)              (None, 1)                 3         
                                                                 
Total params: 560
Trainable params: 560
Non-trainable params: 0
_________________________________________________________________


In [9]:
for i in range(5):
    W, b = model.get_layer(f"layer{i+1}").get_weights()
    print(f"Layer{i+1} : W{W.shape} & b{b.shape}")

Layer1 : W(13, 20) & b(20,)
Layer2 : W(20, 10) & b(10,)
Layer3 : W(10, 5) & b(5,)
Layer4 : W(5, 2) & b(2,)
Layer5 : W(2, 1) & b(1,)


In [10]:
#train model

model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)

model.fit(
    Xt,Yt,            
    epochs=15,
)

Epoch 1/15
7046/7046 [==============================] - 10s 1ms/step - loss: 0.0813
Epoch 2/15
7046/7046 [==============================] - 8s 1ms/step - loss: 0.0691
Epoch 3/15
7046/7046 [==============================] - 9s 1ms/step - loss: 0.0690
Epoch 4/15
7046/7046 [==============================] - 9s 1ms/step - loss: 0.0688
Epoch 5/15
7046/7046 [==============================] - 9s 1ms/step - loss: 0.0686
Epoch 6/15
7046/7046 [==============================] - 8s 1ms/step - loss: 0.0684
Epoch 7/15
7046/7046 [==============================] - 8s 1ms/step - loss: 0.0685
Epoch 8/15
7046/7046 [==============================] - 9s 1ms/step - loss: 0.0682
Epoch 9/15
7046/7046 [==============================] - 9s 1ms/step - loss: 0.0686
Epoch 10/15
7046/7046 [==============================] - 9s 1ms/step - loss: 0.0680
Epoch 11/15
7046/7046 [==============================] - 9s 1ms/step - loss: 0.0681
Epoch 12/15
7046/7046 [==============================] - 8s 1ms/step - loss: 0.0682


### Predictions

In [11]:
xTestN = norm_l(xTest)
predictions = model.predict(xTestN)
print("predictions = \n", predictions)

196/196 [==============================] - 0s 910us/step
predictions = 
 [[0.05855832]
 [0.05855832]
 [0.05855832]
 ...
 [0.05855832]
 [0.05855832]
 [0.05855832]]


In [12]:
yPredicted = (predictions >= 0.5).astype(int)
# print(f"decisions = /n{yPredicted}")
one = 0
for i in range(yPredicted.shape[0]):
    if yPredicted[i,:] == 1 :
        one += 1

print(one)

4352


In [13]:
#Check Accuracy
Matched = 0
for i in range(yPredicted.shape[0]):

    if yPredicted[i,:] == yTest[i,:] :
        Matched += 1

Accuracy = (Matched/yPredicted.shape[0])*100
print(f"Accuracy = {Accuracy} %")  


Accuracy = 98.05205173239662 %
